# Notebook 1 — Ingesta de datos y tipado (FraSoHome)

Este notebook tiene un propósito **formativo**: vamos a **cargar** los archivos CSV origen del caso FraSoHome, realizar una **inspección inicial** (profiling ligero), aplicar **conversiones de tipos** (especial atención a **fechas**) y comprobar la **consistencia básica** de cada dataset (claves primarias, duplicados, referencias entre tablas).

## 0) Preparación del entorno

- Requisitos: `pandas` y `numpy`.
- Asumimos que los CSV están en la  carpeta que definimos en la variable `DATA_DIR`


In [1]:
from __future__ import annotations

import re
import numpy as np
import pandas as pd
from pathlib import Path
from IPython.display import display  # explícito para notebooks

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)
pd.set_option("display.max_colwidth", 60)

DATA_DIR = Path(f"data/raw/")  # <-- ajusta si tus CSV están en otra ruta

EXPECTED_FILES = {
    "crm": "crm.csv",
    "productos": "productos.csv",
    "tiendas": "tiendas.csv",
    "pedidos": "pedidos.csv",
    "lineas_pedido": "lineas_pedido.csv",
    "devoluciones_online": "devoluciones_online.csv",
    "ventas_pos": "ventas_pos.csv",
    "devoluciones_tienda": "devoluciones_tienda.csv",
    "pagos_tienda": "pagos_tienda.csv",
    "stock_diario": "stock_diario.csv",
    # Dataset integrado (opcional en Notebook 1; útil para validar el resultado final)
    "fact_transacciones": "fact_transacciones.csv",
}

missing = [fname for fname in EXPECTED_FILES.values() if not (DATA_DIR / fname).exists()]
if missing:
    print("⚠️ Faltan archivos:", missing)
else:
    print("✅ Todos los archivos esperados están presentes.")

✅ Todos los archivos esperados están presentes.


## 1) Funciones reutilizables (ingesta, perfilado, tipado)

Encapsulamos la lógica en funciones para que puedas reutilizarla con *cualquier* DataFrame.


In [ ]:
# -------------------------
# 1.1 Lectura robusta de CSV
# -------------------------

DEFAULT_NA_VALUES = ["", " ", "NA", "N/A", "NULL", "null", "None", "none", "nan", "NaN"]

def read_csv_raw(path: Path, *, encoding: str = "utf-8", sep: str = ",") -> pd.DataFrame:
    """
    Lee un CSV preservando los valores problemáticos:
    - Carga TODO como string (dtype=str) para NO perder formatos originales.
    - Define na_values comunes para convertir vacíos a NaN.
    """
    df = pd.read_csv(
        path,
        sep=sep,
        encoding=encoding,
        dtype=str,
        na_values=DEFAULT_NA_VALUES,
        keep_default_na=True,
        low_memory=False
    )
    return df

# -------------------------
# 1.2 Perfilado básico
# -------------------------

def basic_overview(df: pd.DataFrame, name: str, head: int = 5) -> None:
    print(f"\n{'='*80}\nDATASET: {name}\n{'='*80}")
    print("Shape:", df.shape)
    print("\nColumnas:", list(df.columns))
    display(df.head(head))
    print("\nDtypes (raw):")
    display(df.dtypes.to_frame("dtype_raw"))
    print("\nNulos por columna (top 15):")
    na = df.isna().sum().sort_values(ascending=False)
    display(na.head(15).to_frame("nulos"))
    print("\nFilas duplicadas exactas:", int(df.duplicated().sum()))

def duplicate_key_report(df: pd.DataFrame, key_cols: list[str], name: str) -> pd.DataFrame:
    """
    Reporta duplicidad de una clave (o clave compuesta).
    Devuelve una tabla con:
    - filas con clave nula (alguna col clave es NaN)
    - número de claves duplicadas
    - número de filas implicadas en duplicados
    """
    key = df[key_cols]
    null_key_rows = int(key.isna().any(axis=1).sum())
    dup_mask = df.duplicated(subset=key_cols, keep=False)
    dup_rows = int(dup_mask.sum())
    dup_keys = int(df.loc[dup_mask, key_cols].dropna().drop_duplicates().shape[0])
    out = pd.DataFrame([{
        "dataset": name,
        "key_cols": ", ".join(key_cols),
        "rows": df.shape[0],
        "null_key_rows": null_key_rows,
        "duplicate_keys": dup_keys,
        "rows_in_duplicates": dup_rows
    }])
    return out

# -------------------------
# 1.3 Normalización / tipado: fechas
# -------------------------

_SP_MONTHS = {
    "enero": "01", "febrero": "02", "marzo": "03", "abril": "04",
    "mayo": "05", "junio": "06", "julio": "07", "agosto": "08",
    "septiembre": "09", "setiembre": "09", "octubre": "10",
    "noviembre": "11", "diciembre": "12"
}

def _normalize_spanish_date_text(s: str) -> str:
    """
    Convierte textos tipo '10 de Enero de 2023' a '2023-01-10'.
    Devuelve el string original si no matchea.
    """
    if s is None:
        return s
    x = str(s).strip()
    if not x:
        return x
    # Quitamos múltiples espacios y ponemos a minúsculas para matchear meses
    x2 = re.sub(r"\s+", " ", x).strip()
    x2_low = x2.lower()

    # patrón: dd de <mes> de yyyy
    m = re.match(r"^(\d{1,2})\s+de\s+([a-záéíóúñ]+)\s+de\s+(\d{2,4})$", x2_low, flags=re.IGNORECASE)
    if not m:
        return x
    dd, month_word, yy = m.group(1), m.group(2), m.group(3)
    month_word = month_word.replace("á", "a").replace("é", "e").replace("í", "i").replace("ó", "o").replace("ú", "u")
    mm = _SP_MONTHS.get(month_word, None)
    if mm is None:
        return x
    # normalizamos año 2 dígitos (heurística)
    if len(yy) == 2:
        yy = "20" + yy  # para el caso educativo, asumimos 20xx
    return f"{yy}-{mm}-{int(dd):02d}"

def parse_datetime_es(series: pd.Series, *, dayfirst: bool = True) -> pd.Series:
    """
    Convierte una serie a datetime de forma robusta:
    - 1ª pasada: pd.to_datetime con inferencia.
    - 2ª pasada: intenta normalizar fechas en texto con meses en español.
    - Si hay ambigüedad, prueba dayfirst True/False y elige la mejor (más parses).
    """
    s = series.astype("string")
    s = s.str.strip()

    # 1) Intento inicial (dayfirst elegido)
    dt1 = pd.to_datetime(s, errors="coerce", dayfirst=dayfirst)

    # 2) Intento alternativo dayfirst
    dt2 = pd.to_datetime(s, errors="coerce", dayfirst=not dayfirst)

    # Elegimos el que parsea más valores
    if dt2.notna().sum() > dt1.notna().sum():
        dt = dt2
    else:
        dt = dt1

    # 3) Normalización de textos en español y re-parse
    needs_help = dt.isna() & s.notna()
    if needs_help.any():
        s_fix = s.copy()
        s_fix.loc[needs_help] = s_fix.loc[needs_help].map(_normalize_spanish_date_text)
        dt_fix_1 = pd.to_datetime(s_fix, errors="coerce", dayfirst=True)
        dt_fix_2 = pd.to_datetime(s_fix, errors="coerce", dayfirst=False)
        dt_fix = dt_fix_1 if dt_fix_1.notna().sum() >= dt_fix_2.notna().sum() else dt_fix_2
        # combinamos (manteniendo los ya parseados)
        dt = dt.fillna(dt_fix)

    return dt

def date_parse_report(df: pd.DataFrame, col: str, *, dayfirst: bool = True, sample_bad: int = 8) -> pd.Series:
    """
    Convierte una columna a datetime y muestra un mini-informe:
    - % de valores parseados
    - ejemplos no parseados (raw)
    """
    parsed = parse_datetime_es(df[col], dayfirst=dayfirst)
    total = len(df)
    ok = int(parsed.notna().sum())
    bad = total - ok
    print(f"🕒 Columna '{col}': {ok}/{total} parseados ({ok/total:.1%}), {bad} NaT")
    if bad > 0:
        bad_examples = df.loc[parsed.isna() & df[col].notna(), col].astype(str).head(sample_bad).tolist()
        print("   Ejemplos NO parseados:", bad_examples)
    return parsed

# -------------------------
# 1.4 Normalización / tipado: numéricos (EUR)
# -------------------------

def parse_numeric(series: pd.Series) -> pd.Series:
    """
    Convierte strings numéricos con formatos mixtos:
    - '€1.234,56' / '1,234.56' / '1234,56' / '1234.56' / '€120.5' / '0'
    a float.
    """
    s = series.astype("string").str.strip()

    # vacíos -> NaN
    s = s.replace({pd.NA: pd.NA, "": pd.NA})

    # quitamos símbolos y texto común
    s = s.str.replace("€", "", regex=False)
    s = s.str.replace("EUR", "", regex=False)
    s = s.str.replace("eur", "", regex=False)

    # quitamos espacios
    s = s.str.replace(r"\s+", "", regex=True)

    def _normalize_one(x: str) -> str:
        if x is None or x is pd.NA:
            return x
        x = str(x)
        if x == "" or x.lower() in ("nan", "none"):
            return ""
        # si tiene coma y punto, decidimos decimal por el separador más a la derecha
        if "," in x and "." in x:
            if x.rfind(",") > x.rfind("."):
                # decimal = coma, miles = punto
                x = x.replace(".", "")
                x = x.replace(",", ".")
            else:
                # decimal = punto, miles = coma
                x = x.replace(",", "")
        elif "," in x and "." not in x:
            # asumimos coma decimal
            x = x.replace(",", ".")
        # si solo punto: ok
        return x

    s_norm = s.map(_normalize_one)
    out = pd.to_numeric(s_norm, errors="coerce")
    return out

# -------------------------
# 1.5 Checks de consistencia (referencias)
# -------------------------

def ref_integrity_report(
    child: pd.DataFrame, child_col: str,
    parent: pd.DataFrame, parent_col: str,
    *, child_name: str, parent_name: str,
    sample: int = 8
) -> pd.DataFrame:
    """
    Calcula cuántos valores de child_col NO aparecen en parent_col (ignorando nulos).
    Devuelve un mini-reporte y muestra ejemplos.
    """
    c = child[child_col].astype("string").str.strip()
    p = parent[parent_col].astype("string").str.strip()

    c_non_null = c.dropna()
    p_set = set(p.dropna().unique().tolist())

    missing_mask = ~c_non_null.isin(p_set)
    missing_count = int(missing_mask.sum())
    total_non_null = int(c_non_null.shape[0])
    pct = (missing_count / total_non_null) if total_non_null else np.nan

    examples = c_non_null.loc[missing_mask].head(sample).tolist()
    if missing_count:
        print(f"🔗 FK check {child_name}.{child_col} -> {parent_name}.{parent_col}: "
              f"{missing_count}/{total_non_null} ({pct:.1%}) NO encontrados. Ejemplos: {examples}")
    else:
        print(f"🔗 FK check {child_name}.{child_col} -> {parent_name}.{parent_col}: OK (0 huérfanos).")

    return pd.DataFrame([{
        "child": child_name,
        "child_col": child_col,
        "parent": parent_name,
        "parent_col": parent_col,
        "non_null_child": total_non_null,
        "missing_refs": missing_count,
        "missing_pct": pct
    }])

# -------------------------
# 1.6 Aplicación de reglas de tipado
# -------------------------

def apply_typing_rules(df: pd.DataFrame, *, date_cols: list[str] | None = None,
                       numeric_cols: list[str] | None = None,
                       int_cols: list[str] | None = None,
                       category_cols: list[str] | None = None) -> pd.DataFrame:
    """
    Aplica tipado básico:
    - Fechas -> datetime64[ns]
    - Numéricos -> float
    - Int (cuando procede) -> Int64 (nullable)
    - Categóricas -> category
    """
    out = df.copy()

    date_cols = date_cols or []
    numeric_cols = numeric_cols or []
    int_cols = int_cols or []
    category_cols = category_cols or []

    for c in date_cols:
        if c in out.columns:
            out[c + "_dt"] = date_parse_report(out, c)
        else:
            print(f"⚠️ date_col '{c}' no existe en df.")

    for c in numeric_cols:
        if c in out.columns:
            out[c + "_num"] = parse_numeric(out[c])
        else:
            print(f"⚠️ numeric_col '{c}' no existe en df.")

    for c in int_cols:
        if c in out.columns:
            out[c + "_int"] = parse_numeric(out[c]).round().astype("Int64")
        else:
            print(f"⚠️ int_col '{c}' no existe en df.")

    for c in category_cols:
        if c in out.columns:
            out[c] = out[c].astype("category")
        # si no existe, lo ignoramos sin ruido

    return out


## 2) Carga de archivos origen

Leemos todos los CSV como *raw strings* para poder inspeccionar y tipar nosotros (y no perder los “errores”).


In [3]:
sources = {name: DATA_DIR / fname for name, fname in EXPECTED_FILES.items()}

dfs = {name: read_csv_raw(path) for name, path in sources.items()}

# Acceso rápido
crm = dfs["crm"]
productos = dfs["productos"]
tiendas = dfs["tiendas"]
pedidos = dfs["pedidos"]
lineas_pedido = dfs["lineas_pedido"]
devoluciones_online = dfs["devoluciones_online"]
ventas_pos = dfs["ventas_pos"]
devoluciones_tienda = dfs["devoluciones_tienda"]
pagos_tienda = dfs["pagos_tienda"]
stock_diario = dfs["stock_diario"]
fact_transacciones = dfs["fact_transacciones"]

print("Datasets cargados:", list(dfs.keys()))


Datasets cargados: ['crm', 'productos', 'tiendas', 'pedidos', 'lineas_pedido', 'devoluciones_online', 'ventas_pos', 'devoluciones_tienda', 'pagos_tienda', 'stock_diario', 'fact_transacciones']


## 3) Inspección inicial (perfilado ligero)

Para cada dataset:
- Dimensiones
- Tipos detectados (raw)
- Nulos
- Duplicados exactos


In [4]:
for name, df in dfs.items():
    basic_overview(df, name)



DATASET: crm
Shape: (103, 20)

Columnas: ['customer_id', 'nombre', 'apellidos', 'email', 'telefono', 'fecha_alta_programa', 'tier_fidelizacion', 'puntos_acumulados', 'fecha_nacimiento', 'genero', 'ciudad', 'provincia', 'codigo_postal', 'pais', 'consentimiento_marketing', 'estado_cliente', 'fecha_baja', 'origen_alta', 'canal_preferido_declarado', 'ultima_actualizacion']


,customer_id,nombre,apellidos,email,telefono,fecha_alta_programa,tier_fidelizacion,puntos_acumulados,fecha_nacimiento,genero,ciudad,provincia,codigo_postal,pais,consentimiento_marketing,estado_cliente,fecha_baja,origen_alta,canal_preferido_declarado,ultima_actualizacion
0,C0001,Bruno,PérezFernández,NaN,+34 662 454 433,23/12/2018,Bronce,3590,25/01/1966,NaN,Sevilla,Sevilla,41001,ES,no,Bloqueado,2024-11-20,NaN,Online,28/06/25 11:38
1,C0002,Bárbara,MartínezRubio,barbara.martinezrubio@example.com,627112365,07/07/2018,Platino,2241,1992-11-03,F,Gijón,Asturias,33201,ES,1,Inactivo,2025-02-16,tienda,Mixto,2025-01-08 01:51:20
2,C0003,Rita,NavarroDíaz,rita.navarrodiaz@example.com,NaN,2024-04-18,Plata,2.073,NaN,F,Córdoba,Córdoba,14001,ES,YES,Activo,NaN,NaN,Mixto,2025-09-28 00:43:46
3,C0004,Moisés,TorresPérez,moises.torresperez@example.com,NaN,2023/09/11,Oro,2073,NaN,No binario,Palma,Illes Balears,07001,ES,0,Activo,NaN,importación,Mixto,02/04/25 07:56
4,C0005,Laura,OrtizGómez,laura.ortizgomez@example.com,707103679,2020/05/17,VIP,6974,1986-09-08,M,Palma,Illes Balears,07001,ES,N,Activo,NaN,online,Mixto,2025-02-17 07:17:42



Dtypes (raw):


,dtype_raw
customer_id,object
nombre,object
apellidos,object
email,object
telefono,object
fecha_alta_programa,object
tier_fidelizacion,object
puntos_acumulados,object
fecha_nacimiento,object
genero,object



Nulos por columna (top 15):


...


Dtypes (raw):


,dtype_raw
fact_id,object
source_system,object
source_file,object
canal_origen,object
canal_movimiento,object
tipo_movimiento,object
es_devolucion,object
fecha_movimiento_raw,object
fecha_movimiento,object
store_id_raw,object



Nulos por columna (top 15):


,nulos
metodo_devolucion,4563
motivo_devolucion,4558
return_id,4522
origen,3312
metodo_pago,2803
order_line_id_original,2766
order_id_original,2708
estado_pedido,2603
cajero_id,2289
caja_id,2288



Filas duplicadas exactas: 0


## 4) Tipado básico (fechas y numéricos)

En este paso:
- Creamos **columnas nuevas** con sufijo `_dt`, `_num`, `_int` para NO destruir el raw.
- Medimos la tasa de parseo (especialmente en fechas).
- Dejamos listas las columnas para el Notebook 2/3 (calidad y limpieza).


In [5]:
# Reglas por dataset (puedes ampliarlas en clase)

crm_t = apply_typing_rules(
    crm,
    date_cols=["fecha_alta_programa", "fecha_nacimiento", "fecha_baja", "ultima_actualizacion"],
    numeric_cols=["puntos_acumulados"],
    category_cols=["tier_fidelizacion", "genero", "ciudad", "provincia", "pais", "consentimiento_marketing",
                   "estado_cliente", "origen_alta", "canal_preferido_declarado"]
)

productos_t = apply_typing_rules(
    productos,
    date_cols=["fecha_alta_catalogo"],
    numeric_cols=["precio_venta", "coste_unitario", "iva_pct", "peso_kg", "largo_cm", "ancho_cm", "alto_cm"],
    category_cols=["categoria", "subcategoria", "marca", "proveedor", "material", "color", "estado_producto"]
)

tiendas_t = apply_typing_rules(
    tiendas,
    date_cols=["fecha_apertura"],
    numeric_cols=["metros_cuadrados", "lat", "lon"],
    category_cols=["tipo_ubicacion", "canal", "ciudad", "provincia", "pais", "estado", "region"]
)

pedidos_t = apply_typing_rules(
    pedidos,
    date_cols=["fecha_pedido", "ultima_actualizacion"],
    numeric_cols=["importe_total", "gastos_envio"],
    category_cols=["moneda", "metodo_pago", "estado_pedido", "origen", "pais_envio"]
)

lineas_pedido_t = apply_typing_rules(
    lineas_pedido,
    numeric_cols=["cantidad", "precio_unitario", "descuento_pct", "descuento_importe", "importe_linea", "iva_pct"]
)

devoluciones_online_t = apply_typing_rules(
    devoluciones_online,
    date_cols=["fecha_devolucion"],
    numeric_cols=["cantidad_devuelta", "importe_reembolsado"],
    category_cols=["motivo_devolucion", "metodo_devolucion", "estado_devolucion", "moneda"]
)

ventas_pos_t = apply_typing_rules(
    ventas_pos,
    date_cols=["fecha_hora"],
    numeric_cols=["cantidad", "precio_unitario", "descuento_pct", "descuento_importe", "importe_linea"],
    category_cols=["store_id", "moneda", "canal", "categoria", "subcategoria"]
)

devoluciones_tienda_t = apply_typing_rules(
    devoluciones_tienda,
    date_cols=["fecha_devolucion"],
    numeric_cols=["cantidad_devuelta", "importe_reembolsado"],
    category_cols=["store_id", "estado_devolucion", "moneda", "metodo_reembolso", "motivo_devolucion", "canal_origen_venta"]
)

pagos_tienda_t = apply_typing_rules(
    pagos_tienda,
    date_cols=["fecha_pago"],
    numeric_cols=["importe_pagado"],
    category_cols=["store_id", "metodo_pago", "moneda", "estado_pago"]
)

stock_diario_t = apply_typing_rules(
    stock_diario,
    date_cols=["fecha"],
    numeric_cols=["stock_cierre", "stock_reservado", "stock_en_transito", "stock_minimo"],
    category_cols=["store_id", "fuente"]
)

# (Opcional) fact integrada — ya viene con columnas parseadas, pero inspeccionamos fechas clave:
fact_t = apply_typing_rules(
    fact_transacciones,
    date_cols=["fecha_movimiento_raw", "fecha_movimiento"],
    numeric_cols=["cantidad_signed", "importe_signed_num", "importe_abs_num", "coste_total_num", "margen_bruto_num"]
)


C:\Users\anton\AppData\Local\Temp\ipykernel_14408\1688482174.py:116: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dt2 = pd.to_datetime(s, errors="coerce", dayfirst=not dayfirst)
C:\Users\anton\AppData\Local\Temp\ipykernel_14408\1688482174.py:130: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dt_fix_2 = pd.to_datetime(s_fix, errors="coerce", dayfirst=False)
C:\Users\anton\AppData\Local\Temp\ipykernel_14408\1688482174.py:116: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  dt2 = pd.to_datetime(s, errors="coerce", dayfirst=not dayfirst)
C:\Users\anton\AppData\Local\Temp\ipykernel_14408\1688482174.py:130: UserWarning: Parsing dates in %d/%m/%Y fo

🕒 Columna 'fecha_alta_programa': 22/103 parseados (21.4%), 81 NaT
   Ejemplos NO parseados: ['2024-04-18', '2023/09/11', '2020/05/17', '2022-10-01', '2025-03-28', '27 de Agosto de 2020', '13 de Marzo de 2019', '2019-09-10']
🕒 Columna 'fecha_nacimiento': 29/103 parseados (28.2%), 74 NaT
   Ejemplos NO parseados: ['1992-11-03', '1986-09-08', '1960-05-15', '1959-01-17', '2005-04-12', '1959-12-04', '1997-08-19', '1996-09-11']
🕒 Columna 'fecha_baja': 33/103 parseados (32.0%), 70 NaT
   Ejemplos NO parseados: ['07/11/2024', '05/09/2023', '13/08/2024', '17/10/2025', '20/02/2025', '2024/03/15', '04/09/2024', '2024/08/12']
🕒 Columna 'ultima_actualizacion': 103/103 parseados (100.0%), 0 NaT
🕒 Columna 'fecha_alta_catalogo': 100/101 parseados (99.0%), 1 NaT
   Ejemplos NO parseados: ['2023/02/05']
🕒 Columna 'fecha_apertura': 6/8 parseados (75.0%), 2 NaT
   Ejemplos NO parseados: ['15/07/2017', '2020/09/30']
🕒 Columna 'fecha_pedido': 461/656 parseados (70.3%), 195 NaT
   Ejemplos NO parseados: ['20

C:\Users\anton\AppData\Local\Temp\ipykernel_14408\1688482174.py:116: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt2 = pd.to_datetime(s, errors="coerce", dayfirst=not dayfirst)
C:\Users\anton\AppData\Local\Temp\ipykernel_14408\1688482174.py:129: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_fix_1 = pd.to_datetime(s_fix, errors="coerce", dayfirst=True)
C:\Users\anton\AppData\Local\Temp\ipykernel_14408\1688482174.py:130: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_fix_2 = pd.to_datetime(s_fix, errors="coerce", dayfirst=False)
C:\Users\anton\AppData\Local\Temp\ipyker

🕒 Columna 'fecha_hora': 2351/2521 parseados (93.3%), 170 NaT
   Ejemplos NO parseados: ['20 de septiembre de 2025 05:26', '20 de septiembre de 2025 05:26', '26 de diciembre de 2025 10:59', '26 de diciembre de 2025 10:59', '13 de mayo de 2025 07:36', '13 de mayo de 2025 07:36', '15 de febrero de 2025 14:51', '10 de abril de 2025 19:08']
🕒 Columna 'fecha_devolucion': 170/211 parseados (80.6%), 41 NaT
   Ejemplos NO parseados: ['14 de septiembre de 2025 08:25', '5 de diciembre de 2025 04:21', '3 de noviembre de 2025 20:51', '5 de febrero de 2025 22:58', '2 de agosto de 2025 22:05', '15 de julio de 2025 22:59', '17 de febrero de 2025 13:34', '14 de enero de 2026 10:00']
🕒 Columna 'fecha_pago': 549/1646 parseados (33.4%), 1097 NaT
   Ejemplos NO parseados: ['28/09/25 10:29', '07/04/25 00:50', '17/05/25 19:52', '28/02/2025 18:54', '02/09/25 02:42', '19/08/25 02:40', '05/09/2025 00:00', '30/03/2025']
🕒 Columna 'fecha': 41/1940 parseados (2.1%), 1899 NaT
   Ejemplos NO parseados: ['2025-09-01'

C:\Users\anton\AppData\Local\Temp\ipykernel_14408\1688482174.py:113: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  dt1 = pd.to_datetime(s, errors="coerce", dayfirst=dayfirst)
C:\Users\anton\AppData\Local\Temp\ipykernel_14408\1688482174.py:129: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  dt_fix_1 = pd.to_datetime(s_fix, errors="coerce", dayfirst=True)


## 5) Consistencia básica: claves y referencias

En esta sección comprobamos:
- Claves (PK): duplicados y nulos (esperamos “fallos” por el diseño del caso).
- Integridad referencial (FK): referencias huérfanas entre tablas.

Esto no es una “limpieza”, es un **diagnóstico** inicial.


In [6]:
# 5.1 Duplicidad de claves esperadas (PK)

reports = []
reports.append(duplicate_key_report(crm, ["customer_id"], "crm"))
reports.append(duplicate_key_report(productos, ["product_id"], "productos"))
reports.append(duplicate_key_report(tiendas, ["store_id"], "tiendas"))

reports.append(duplicate_key_report(pedidos, ["order_id"], "pedidos"))
reports.append(duplicate_key_report(lineas_pedido, ["order_line_id"], "lineas_pedido"))
reports.append(duplicate_key_report(devoluciones_online, ["return_id"], "devoluciones_online"))

reports.append(duplicate_key_report(ventas_pos, ["ticket_line_id"], "ventas_pos"))
reports.append(duplicate_key_report(devoluciones_tienda, ["return_id"], "devoluciones_tienda"))
reports.append(duplicate_key_report(pagos_tienda, ["payment_id"], "pagos_tienda"))

reports.append(duplicate_key_report(stock_diario, ["fecha", "store_id", "product_id"], "stock_diario"))

pk_report = pd.concat(reports, ignore_index=True)
display(pk_report)

# 5.2 Integridad referencial (FK) — usando columnas RAW (sin estandarizar aún)
fk_reports = []

# Pedidos -> CRM
fk_reports.append(ref_integrity_report(pedidos, "customer_id", crm, "customer_id",
                                      child_name="pedidos", parent_name="crm"))

# Líneas -> Pedidos y Productos
fk_reports.append(ref_integrity_report(lineas_pedido, "order_id", pedidos, "order_id",
                                      child_name="lineas_pedido", parent_name="pedidos"))
fk_reports.append(ref_integrity_report(lineas_pedido, "product_id", productos, "product_id",
                                      child_name="lineas_pedido", parent_name="productos"))

# Devoluciones online -> Pedidos, Líneas, Productos
fk_reports.append(ref_integrity_report(devoluciones_online, "order_id", pedidos, "order_id",
                                      child_name="devoluciones_online", parent_name="pedidos"))
fk_reports.append(ref_integrity_report(devoluciones_online, "order_line_id", lineas_pedido, "order_line_id",
                                      child_name="devoluciones_online", parent_name="lineas_pedido"))
fk_reports.append(ref_integrity_report(devoluciones_online, "product_id", productos, "product_id",
                                      child_name="devoluciones_online", parent_name="productos"))

# POS ventas -> Productos, Tiendas, CRM (cuando hay customer_id)
fk_reports.append(ref_integrity_report(ventas_pos, "product_id", productos, "product_id",
                                      child_name="ventas_pos", parent_name="productos"))
fk_reports.append(ref_integrity_report(ventas_pos, "store_id", tiendas, "store_id",
                                      child_name="ventas_pos", parent_name="tiendas"))
fk_reports.append(ref_integrity_report(ventas_pos, "customer_id", crm, "customer_id",
                                      child_name="ventas_pos", parent_name="crm"))

# POS devoluciones -> Ventas POS, Productos, Tiendas, CRM
fk_reports.append(ref_integrity_report(devoluciones_tienda, "ticket_id_original", ventas_pos, "ticket_id",
                                      child_name="devoluciones_tienda", parent_name="ventas_pos"))
fk_reports.append(ref_integrity_report(devoluciones_tienda, "ticket_line_id_original", ventas_pos, "ticket_line_id",
                                      child_name="devoluciones_tienda", parent_name="ventas_pos"))
fk_reports.append(ref_integrity_report(devoluciones_tienda, "product_id", productos, "product_id",
                                      child_name="devoluciones_tienda", parent_name="productos"))
fk_reports.append(ref_integrity_report(devoluciones_tienda, "store_id", tiendas, "store_id",
                                      child_name="devoluciones_tienda", parent_name="tiendas"))
fk_reports.append(ref_integrity_report(devoluciones_tienda, "customer_id", crm, "customer_id",
                                      child_name="devoluciones_tienda", parent_name="crm"))
fk_reports.append(ref_integrity_report(devoluciones_tienda, "order_id_original", pedidos, "order_id",
                                      child_name="devoluciones_tienda", parent_name="pedidos"))

# Pagos -> Ventas POS y Tiendas
fk_reports.append(ref_integrity_report(pagos_tienda, "ticket_id", ventas_pos, "ticket_id",
                                      child_name="pagos_tienda", parent_name="ventas_pos"))
fk_reports.append(ref_integrity_report(pagos_tienda, "store_id", tiendas, "store_id",
                                      child_name="pagos_tienda", parent_name="tiendas"))

# Stock -> Productos y Tiendas
fk_reports.append(ref_integrity_report(stock_diario, "product_id", productos, "product_id",
                                      child_name="stock_diario", parent_name="productos"))
fk_reports.append(ref_integrity_report(stock_diario, "store_id", tiendas, "store_id",
                                      child_name="stock_diario", parent_name="tiendas"))

fk_report = pd.concat(fk_reports, ignore_index=True)
display(fk_report.sort_values(["missing_pct", "missing_refs"], ascending=[False, False]))


,dataset,key_cols,rows,null_key_rows,duplicate_keys,rows_in_duplicates
0,crm,customer_id,103,0,2,4
1,productos,product_id,101,0,1,2
2,tiendas,store_id,8,0,0,0
3,pedidos,order_id,656,0,4,8
4,lineas_pedido,order_line_id,1949,0,19,39
5,devoluciones_online,return_id,224,0,5,10
6,ventas_pos,ticket_line_id,2521,0,40,80
7,devoluciones_tienda,return_id,211,0,5,10
8,pagos_tienda,payment_id,1646,0,15,30
9,stock_diario,"fecha, store_id, product_id",1940,6,29,58


🔗 FK check pedidos.customer_id -> crm.customer_id: 66/493 (13.4%) NO encontrados. Ejemplos: ['C9999', 'C9999', 'CX123', 'C9999', 'CX123', 'C0101', 'CX123', 'c0011']
🔗 FK check lineas_pedido.order_id -> pedidos.order_id: 5/1946 (0.3%) NO encontrados. Ejemplos: ['O-202499999', 'O-XYZ', 'O-XYZ', 'O-XYZ', 'O-XYZ']
🔗 FK check lineas_pedido.product_id -> productos.product_id: 129/1918 (6.7%) NO encontrados. Ejemplos: ['p1000', 'p1097', 'p10X1', 'p1082', 'p1060', 'p10X1', 'P9999', 'SKU-9999']
🔗 FK check devoluciones_online.order_id -> pedidos.order_id: 3/222 (1.4%) NO encontrados. Ejemplos: ['O-209900001', 'O-209900001', 'O-209900001']
🔗 FK check devoluciones_online.order_line_id -> lineas_pedido.order_line_id: 3/214 (1.4%) NO encontrados. Ejemplos: ['OL-00000000', 'OL-00000000', 'OL-00000000']
🔗 FK check devoluciones_online.product_id -> productos.product_id: 28/221 (12.7%) NO encontrados. Ejemplos: ['p10X1', 'p10X1', 'P9999', 'p1029', 'p1081', 'SKU-9999', 'p1020', 'SKU-9999']
🔗 FK check ven

,child,child_col,parent,parent_col,non_null_child,missing_refs,missing_pct
13,devoluciones_tienda,customer_id,crm,customer_id,139,33,0.237410
14,devoluciones_tienda,order_id_original,pedidos,order_id,53,10,0.188679
11,devoluciones_tienda,product_id,productos,product_id,207,30,0.144928
8,ventas_pos,customer_id,crm,customer_id,1402,200,0.142653
0,pedidos,customer_id,crm,customer_id,493,66,0.133874
5,devoluciones_online,product_id,productos,product_id,221,28,0.126697
6,ventas_pos,product_id,productos,product_id,2521,231,0.091630
12,devoluciones_tienda,store_id,tiendas,store_id,208,18,0.086538
2,lineas_pedido,product_id,productos,product_id,1918,129,0.067258
9,devoluciones_tienda,ticket_id_original,ventas_pos,ticket_id,168,10,0.059524


## 6) Checks rápidos específicos de fechas (formativo)

Queremos detectar:
- Fechas futuras o fuera de rango
- Columnas que no parsean bien
- Valores “raros” que habrá que normalizar en limpieza


In [7]:
def date_range_check(df_typed: pd.DataFrame, dt_col: str, name: str,
                     min_date: str = "2015-01-01", max_date: str = "2026-12-31") -> None:
    if dt_col not in df_typed.columns:
        print(f"⚠️ {name}: no existe {dt_col}")
        return
    s = df_typed[dt_col]
    s = s.dropna()
    if s.empty:
        print(f"⚠️ {name}: {dt_col} está vacío tras parseo")
        return
    min_dt = pd.to_datetime(min_date)
    max_dt = pd.to_datetime(max_date)

    out_low = int((s < min_dt).sum())
    out_high = int((s > max_dt).sum())
    print(f"📅 {name}.{dt_col}: min={s.min()}, max={s.max()} | <{min_date}: {out_low} | >{max_date}: {out_high}")

# Ejemplos
date_range_check(pedidos_t, "fecha_pedido_dt", "pedidos")
date_range_check(ventas_pos_t, "fecha_hora_dt", "ventas_pos")
date_range_check(devoluciones_online_t, "fecha_devolucion_dt", "devoluciones_online")
date_range_check(stock_diario_t, "fecha_dt", "stock_diario")
date_range_check(tiendas_t, "fecha_apertura_dt", "tiendas")


📅 pedidos.fecha_pedido_dt: min=2017-01-19 12:00:00, max=2030-01-13 08:59:00 | <2015-01-01: 0 | >2026-12-31: 1
📅 ventas_pos.fecha_hora_dt: min=2024-08-28 13:05:00, max=2026-08-29 06:42:00 | <2015-01-01: 0 | >2026-12-31: 0
📅 devoluciones_online.fecha_devolucion_dt: min=2024-02-01 03:21:03, max=2031-07-01 10:00:00 | <2015-01-01: 0 | >2026-12-31: 3
📅 stock_diario.fecha_dt: min=2025-09-01 03:05:24, max=2025-10-31 09:15:03 | <2015-01-01: 0 | >2026-12-31: 0
📅 tiendas.fecha_apertura_dt: min=2015-11-01 00:00:00, max=2027-05-01 00:00:00 | <2015-01-01: 0 | >2026-12-31: 1


## 7) Siguiente paso (Notebook 2)

Con esta primera ingesta ya sabemos:

1. Qué columnas hay y cómo vienen tipadas **en crudo**.
2. Qué columnas de **fecha** y **número** requieren normalización.
3. Dónde hay **duplicados** y **referencias huérfanas**.

En el Notebook 2 (Perfilado + Data Quality Report) usaremos estos resultados para:
- priorizar problemas,
- decidir reglas de limpieza,
- definir el plan de estandarización (IDs, fechas, importes, categorías).
